# Spatial analysis of Chicago burglary crime patterns 

## 1.0 Introduction and analysis context

word count: ~ 3000

### 1.1 Context and literature review

The continuous digitisation of police recorded crime statistics enables researchers and authorities to probe urban crime in depth. There are several related studies from different perspectives and using various computational techniques. From an academic and governing angle, spatial analysis of criminal activities plays an essential role in criminological research and policymaking (Andresen, 2005). Spatial analysis techniques, particularly DBSCAN,  help law enforcement safeguard crime-prone areas (Thakur, 2020). A spatial analysis of the burglary crime incidents could help policymakers allocate public and human resources to where they can be most effective. Relocating existing resources to crime hotspots identified by DBSCAN can have a positive investment return (Wheeler and Reuter, 2020).

Spatial patterns of the burglary that this project found can be associated with other local socio-economic factors to determine which kind of neighbourhoods are prone to burglaries (Craglia, Haining and Wiles, 2000). Several case studies show the link between local socio-economic factors and the crime rate. A significant correlation was found between the unemployment rate and crimes in an analysis of US data from 1974-2000 (Lin, 2008). Global datasets suggest that rising educational attainment leads to a reduction in criminal activities. One extra year's school education decreases burglary and larceny by around six per cent (Hjalmarsson & Lochner, 2012). One research of the City of Toronto finds crime hotspots in low-income areas while high in-come communities suffer a much lower crime rate (Charron, 2009). This project uses Chicago socio-economic datasets from the heartland alliance to build a model to predict crime rates in different communities.

### 1.2 Project goals

This project aims to provide a statistical spatial analysis of burglary crime incidents in Chicago and related visualisations. 
Appropriate clustering methods will identify clusters of crime incidents and administrative regions with a high burglary rate.
The underlaying socio-economic factors that correlate with high crime risk will be investigated by regression methods.

Goals bullet points and their potential real-world impacts:<br>

| Tasks | Task Categories | Real-world impacts |
| :---        |    :----  |          :--- |
| Identification of burglary crime clusters | Classification of point data | Crime control and preventation, a more efficient use of the police force |
| Correlate socio-economic factors with burglary rate | Regression | Prediction of high-burglary-rate areas |

These tasks can answer two questions: <br> 1. Where should resources (e.g. new surveillance systems, more frequent patrols and more police assistance booths) be allocated to reduce the burglary crime rate in Chicago? <br>
2. Depending on socio-economic data, can the authority act proactively to prevent burglary crime?

### 1.3 Data source 

The complete dataset of Chicago crime incidents from 2001 to the present is available through Chicago Data Portal. However, due to the scope of this analysis and the dataset's massive nature (over 7.5 million rows in March 2022), a filter is essential for obtaining a suitable subset, which contains burglary records only, and a swift data preprocessing and validation section. One way to implement such a filter is using a python package called sodapy, which is not available inside the SDS computing environment. An example is here: [https://github.com/NijunJiang/CASA0006_2022/blob/main/data_query.py]. The example python code needs an app token (registration with Chicago Data Portal required) and one line of NoSQL query to run. This project applies the 2019 dataset since the latest census data of Chicago is from 2019, and the 2020 global pandemic may introduce atypical variables.

Chicago crimes -2001 to Present: https://data.cityofchicago.org/Public-Safety/Crimes-2001-to-Present/ijzp-q8t2

The web page above includes metadata and column descriptions. Two columns, 'Latitude' and 'Longitude', make the geocoding of the point crime events possible.

The Data Portal offers shapefiles of Chicago wards. This project utilised shapefiles to provide an intuitive map of urban burglary density and risk.

Chicago Boundaries - Wards (2015 - ): https://data.cityofchicago.org/Facilities-Geographic-Boundaries/Boundaries-Wards-2015-/sp34-6z76

The heartland alliance provides a range of community level census (socio-economic and population) datasets.

Chicago communities datasets: https://www.heartlandalliance.org/heartland-alliance/research-and-policy/data-reports/chicago-data-dashboards/


This notebook merges the example Chicago crime dataset with spatial features of Chicago communities and socio-economic datasets to provide a more comprehensive analysis and visualisation beyond a simple point data pattern study.

### 1.4 Package requirement

In [ ]:
import contextily as cx
import geopandas as gpd
import geopy.distance
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import statsmodels.api as sm
from IPython.display import HTML, display
from matplotlib_scalebar.scalebar import ScaleBar
from shapely.geometry import Point, Polygon
from sklearn import metrics
from sklearn.cluster import DBSCAN
from sklearn.feature_selection import RFE
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import (
    GridSearchCV,
    KFold,
    cross_val_score,
    train_test_split,
)
from sklearn.neighbors import BallTree
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import MinMaxScaler, PolynomialFeatures, scale
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant

## 2. Data preparation

### 2.1 Data Input

In [ ]:
BURGLARY_2019 = pd.read_csv(
    "https://raw.githubusercontent.com/NijunJiang/CASA0006_2022/main/BURGLARY_2019.csv"
)  # burglary records of Chicago 2019

chicago_community_pop = pd.read_csv(
    "https://raw.githubusercontent.com/NijunJiang/CASA0006_2022/main/chicago_community_population.CSV"
)  # population of each Chicago commmunity

chicago_community = gpd.read_file(
    "/vsicurl/https://raw.githubusercontent.com/NijunJiang/CASA0006_2022/main/geo_export_d1f0aa30-5958-4286-841c-d61250028149.shp"
)  # the shapefile of Chicago communities

median_household_income = pd.read_csv(
    "https://raw.githubusercontent.com/NijunJiang/CASA0006_2022/main/MHI_data_average_median_household_income.csv"
)  # median household income of each Chicago commmunity

education = pd.read_csv(
    "https://raw.githubusercontent.com/NijunJiang/CASA0006_2022/main/bach_data.csv"
)  # percentage of population with a college degree of each Chicago commmunity

unemply = pd.read_csv(
    "https://raw.githubusercontent.com/NijunJiang/CASA0006_2022/main/unemply_data.csv"
)  # overall unemployment rate of each Chicago community

community_reference = pd.read_csv(
    "https://raw.githubusercontent.com/NijunJiang/CASA0006_2022/main/community_reference.csv"
)  # reference table for community name and number to reorder community socio-economics data

# if data is not retrievable, please visit https://github.com/NijunJiang/CASA0006_2022 directly or use offline dataset that is attached with the submission

One example row of the crime record:

In [ ]:
BURGLARY_2019.head(1)

In [ ]:
chicago_community_pop
chicago_community_pop.rename(
    columns={"No.": "community number"}, inplace=True
)  # change the column name to ensure consistent column name between dataframes
chicago_community_pop.head()

In [ ]:
community_type = ["community_dummy"] * len(chicago_community.index)
chicago_community[
    "community_dummy"
] = community_type  # nummpy value for plotting; ensure polygons have the same colour

chicago_community = chicago_community.rename(
    columns={"area_numbe": "community number"}
)  # change the column name to ensure consistent column name between dataframes
chicago_community.head(1)

### 2.2 Data Preprocessing and Validation

Previous sections loaded datasets that this project will analyse. However, several data preprocessing steps should proceed before further analysis. The first step is data cleaning. In the 2019 dataset, there are over nine thousand rows of burglary records. To ensure a successful implementation of spatial analysis techniques (e.g. DBSCAN), filtering out rows that contain NULL, N/A, or zero values in the geographic location columns (e.g. latitude and longitude columns) is necessary. 

In [ ]:
# Do not rerun the section unless restarting the kernel; otherwise the row number difference before and after the cleaning will not show
# Completeness check
before_clearning_size = BURGLARY_2019.shape[0]
BURGLARY_2019 = BURGLARY_2019[BURGLARY_2019["latitude"].notna()]
BURGLARY_2019 = BURGLARY_2019[
    BURGLARY_2019["longitude"].notna()
]  # make sure lat and lon values are valid
after_clearning_size = BURGLARY_2019.shape[0]
print(
    "Before the first step of data cleaning, the 2019 dataset has "
    + str(before_clearning_size)
    + " rows."
    + " After cleaning, there are "
    + str(after_clearning_size)
    + " rows remaining"
)

It is also possible that there are duplicate incidents. The section below will drop potential duplicate rows based on three columns: case_number, id and location.

In [ ]:
# Duplicate identification and elimination
before_clearning_size2 = BURGLARY_2019.shape[0]
BURGLARY_2019.drop_duplicates(subset=["id", "case_number", "location"])
after_clearning_size2 = BURGLARY_2019.shape[0]
print(
    "Before the second step of data cleaning, the 2019 dataset has "
    + str(before_clearning_size2)
    + " rows"
    + " After cleaning, there are "
    + str(after_clearning_size2)
    + " rows remaining"
)

Fortunately, the Data Fulfillment and Analysis Section of the Chicago Police Department did not record any duplicate cases in 2019. The next step is data type checking. We require that location information is in numerical forms, such as float.

In [ ]:
print("Column Latitude data type: " + str(BURGLARY_2019["latitude"].dtypes))
print("Column Longitude data type: " + str(BURGLARY_2019["longitude"].dtypes))

The data type of the two essential columns is, as expected, the float number. The final step of preprocessing should be outlier detection. However, due to the nature (official police records) of this 2019 dataset, it is unlikely that any data point is random. All rows have their wards and community numbers, which means removing isolated cases (outliers in a statistical sense) will reduce the burglary rate in administrative areas and cause bias. While applying DBSCAN, the algorithm will automatically disregard incidents that do not belong to any clusters. This project will not use statistical methods to detect and remove outlier events at the preprocessing stage.

### 2.3 Data Munging

There is one row of the burglary case record in the section below. Twenty-three columns provide a rich range of information, but not all of them are essential ingredients for a spatial study. Among them, four columns: ward, community area, latitude and longitude, will take part in the spatial analysis. Neither regression nor classification will use all remaining descriptive columns.

In [ ]:
BURGLARY_2019.head(1)

Important metadata (see more details in subheading "Columns in this Dataset" at https://data.cityofchicago.org/Public-Safety/Crimes-2001-to-Present/ijzp-q8t2):


|column name | column description |
| :---        |    :----  |
|id|Unique ID for identifing the record.|
|date|Date when the incident occurred (could be an estimate).|
|ward|The ward (City Council district) where the incident occurred. 
|community Area | community area where the incident occurred. Chicago has 77 community areas.
|latitude|The latitude of the location where the incident occurred. |
|longitude|The longitude of the location where the incident occurred.|

In [ ]:
# Removal of superfluous columns
BURGLARY_2019_spatial = BURGLARY_2019[
    ["id", "ward", "community_area", "latitude", "longitude"]
]  # descriptive columns such as fbi_code and arrest have no impact on the spatial analysis

After reducing the complete dataset to a dataset with only municipal locations and coordinates in WSG84, pivot tables can provide an overview of the burglary case distribution across different levels of Chicago municipal districts.

In [ ]:
# pivot tables initialisation
ward_count = BURGLARY_2019_spatial["ward"].value_counts()
community_count = BURGLARY_2019_spatial["community_area"].value_counts()

In [ ]:
# override CSS to make multiple dataframes be able to display together
# https://softhints.com/display-two-pandas-dataframes-side-by-side-jupyter-notebook/
css = """
.output {
    flex-direction: row;
}
"""

HTML("<style>{}</style>".format(css))

In [ ]:
# pivot tables sorting and display
community_count = community_count.sort_index()
community_count_frame = pd.DataFrame(
    {
        "community number": community_count.index,
        "burglary count 2019": community_count.values,
    }
)

ward_count = ward_count.sort_index()
ward_count_frame = pd.DataFrame(
    {"ward": ward_count.index, "burglary count 2019": ward_count.values}
)

display(community_count_frame.head())
display(ward_count_frame.head())  # pivot tables to provide summary information

In the first five districts (ward or community), we can see the variation in incident numbers between different areas.

### 2.4 Data Merging

Given that community population data and total burglary incident numbers within communities are available, we can calculate the burglary rate (incidents per unit of population). The reason for choosing community-level analysis rather than ward is the ward is a political district, and ward boundaries change to reflect population shifts. Community borders, in contrast, are stable over time

In [ ]:
crime_rate_2019_frame = pd.merge(
    community_count_frame, chicago_community_pop, on="community number"
)  # merge population data into the community info dataframe


crime_rate_2019_frame["incident rate per 1000 people"] = crime_rate_2019_frame[
    "burglary count 2019"
] / (
    crime_rate_2019_frame["Population"] / 1000
)  # new column, incident rate per 1000 people, based on two existing columns: burglary count 2019 and population

crime_rate_2019_frame

Append burglary statistical data to the community dataframe.

In [ ]:
chicago_community["community number"] = chicago_community["community number"].astype(
    int
)

chicago_community_sorted = chicago_community.sort_values(by=["community number"])
chicago_community_with_crime_data = pd.merge(
    chicago_community_sorted, crime_rate_2019_frame, on="community number"
)
chicago_community_with_crime_data.head(1)

Append socio-economic factors to the burglary statistical dataframe.

In [ ]:
# reorder dataframe indexing to make sure index is correlated with community number
unemply = unemply.set_index("Geography1")
unemply = unemply.reindex(index=community_reference["Name"])
unemply = unemply.reset_index()
unemply = unemply[["Name", "Estimate (copy)"]]

education = education.set_index("Geography1")
education = education.reindex(index=community_reference["Name"])
education = education.reset_index()
education = education[["Name", "Estimate (copy)"]]

median_household_income = median_household_income.set_index("Geography1")
median_household_income = median_household_income.reindex(
    index=community_reference["Name"]
)
median_household_income = median_household_income.reset_index()
median_household_income = median_household_income[["Name", "Estimate (copy)"]]

In [ ]:
unemply.rename(columns={"Name": "community"}, inplace=True)
education.rename(columns={"Name": "community"}, inplace=True)
median_household_income.rename(columns={"Name": "community"}, inplace=True)

In [ ]:
unemply["community"] = unemply["community"].str.upper()
education["community"] = education["community"].str.upper()
median_household_income["community"] = median_household_income["community"].str.upper()

In [ ]:
unemply.rename(columns={"Estimate (copy)": "unemployment rate"}, inplace=True)
education.rename(columns={"Estimate (copy)": "degree percentage"}, inplace=True)
median_household_income.rename(
    columns={"Estimate (copy)": "median household income"}, inplace=True
)

In [ ]:
chicago_community_with_crime_data = pd.merge(
    chicago_community_with_crime_data, unemply, on="community"
)

chicago_community_with_crime_data = pd.merge(
    chicago_community_with_crime_data, education, on="community"
)

chicago_community_with_crime_data = pd.merge(
    chicago_community_with_crime_data, median_household_income, on="community"
)

In [ ]:
chicago_community_with_crime_data.head(1)

## 3. Exploratory Data Analysis

### 3.1 Crime Map

With the help from the package geopandas, this section plots the visualisation of locations of burglary crime over the Chicago map. 

In [ ]:
crs = {
    "init": "epsg:4326"
}  # coordinate reference system for the latitude and longitude


geometry = [
    Point(xy)
    for xy in zip(BURGLARY_2019_spatial["longitude"], BURGLARY_2019_spatial["latitude"])
]


geodata = gpd.GeoDataFrame(
    BURGLARY_2019_spatial, crs=crs, geometry=geometry
)  # burglary incident point data


fig, ax = plt.subplots(figsize=(10, 10))
ax.set_xlabel("Longitude")
ax.set_ylabel("Latitude")
ax.set_title("Recorded burglary incidents across the Chicago")

chicago_community.plot(
    ax=ax, legend=True, column="community_dummy", categorical=True, alpha=0.6
)
ax.legend()
ax.scatter(
    BURGLARY_2019_spatial["longitude"],
    BURGLARY_2019_spatial["latitude"],
    label="burglary incident",
    s=1,
    c="red",
)
ax.legend()
cx.add_basemap(ax, crs=chicago_community.crs)  # add basemap
# some ideas from this article: https://www.linkedin.com/pulse/geopandas-plotting-data-points-map-using-python-r%C3%A9gis-nisengwe/

With the help of the package geopandas, this section plots the visualisation of locations of crime events over the Chicago map. This representation of the raw data provides an intuitive impression that the burglary pattern does not have an even distribution across the Chicago urban area. Four southeast districts and a few northwest areas have fewer cases than the remaining communities. At both sides of the highway 55, there are two strips of low-burglary-rate zone.

Plots of pivot tables:

In [ ]:
community_count_frame.plot.bar(
    x="community number",
    figsize=(10, 10),
    xlabel="community number index",
    ylabel="number of burglary incident 2019",
    title="Community level burglary total number",
)

In [ ]:
ward_count_frame.plot.bar(
    x="ward",
    figsize=(10, 10),
    xlabel="ward number index",
    ylabel="number of burglary incident 2019",
    title="Ward level burglary total number",
)

Both diagrams (Ward level burglary total number and Community level burglary total number) illustrate the significant difference in the administrative units' crime statistics.

### 3.2 Community-level Crime Rate 

In [ ]:
fig, ax = plt.subplots(figsize=(10, 7))
chicago_community_with_crime_data.plot(
    column="incident rate per 1000 people",
    ax=ax,
    legend=True,
    legend_kwds={"label": "incident rate per 1000 people"},
)
chicago_community.apply(  # add community number to the polygon
    lambda x: ax.annotate(
        text=x["community number"],
        xy=x.geometry.centroid.coords[0],
        ha="center",
        color="#ee8d18",
    ),
    axis=1,
)

plt.plot([], [], " ", label="70: community number")
legend = plt.legend()
plt.setp(legend.get_texts(), color="#ee8d18")
ax.set_xlabel("Longitude")
ax.set_ylabel("Latitude")
ax.set_title("Incident rate per 1000 people (community level)")

Southern communities (communities around and including communities 69) have a higher burglary rate than their northern counterparts.

In [ ]:
fig, ax = plt.subplots(figsize=(10, 7))
chicago_community_with_crime_data.plot(
    column="burglary count 2019",
    ax=ax,
    legend=True,
    legend_kwds={"label": "burglary count 2019"},
)
chicago_community.apply(
    lambda x: ax.annotate(
        text=x["community number"],
        xy=x.geometry.centroid.coords[0],
        ha="center",
        color="#ee8d18",
    ),
    axis=1,
)

plt.plot([], [], " ", label="70: community number")
legend = plt.legend()
plt.setp(legend.get_texts(), color="#ee8d18")
ax.set_xlabel("Longitude")
ax.set_ylabel("Latitude")
ax.set_title("Total 2019 burglary count in each Chicago community")

The spatial distribution of communities with higher total burglary crime records is less concentrated. Community 43 has the highest number among all others. Communities adjacent to the community 24 and 71 tend to have a higher total count than the other communities.

### 3.3 Community-level Summary Statistics

In [ ]:
chicago_community_with_crime_data[["burglary count 2019"]].describe()

On average, during 2019, around 128 burglary incidents happened in each community.

In [ ]:
chicago_community_with_crime_data[["incident rate per 1000 people"]].describe()

The average incident rate per 1000 people (cases per 1000 people) is near 3.86.

## 3. Methodology

### 3.1 DBSCAN (clustering)

A range of clustering methods, including the DBSCAN algorithm, K - Nearest Neighbor and Artificial Neural Network, were applicable to crime pattern mapping (Mungekar et al., 2021). This project will not evaluate different methods' performance. The choice of the approach is based on a study analysing six Indian cities. During the research, DBSCAN outperforms K-Means clustering and Agglomerative hierarchical clustering in terms of recall and precision (Sivaranjani, Sivakumari and Aasha, 2016).

DBSCAN (density-based spatial clustering of applications with noise) groups crowded points into groups and determines whether other points belong to any clusters. DBSCAN is an unsupervised learning method that requires two hyperparameters (epsilon, which determines the Euclidean searching radius of the data point; minPoints, which is the minimum number of other data points inside the search radius for the central data point to become a core point) to initialise. Within the search radius, if a point finds n (greater than zero but smaller than minPoints) other points, the point becomes a border point. The DBCAN algorithm first identifies and connects core points to form clusters and then connects border points to suitable clusters. A point that finds no other point within the search radius becomes a noise point. (Ester et al., 1996)

DBSCAN is an unsupervised learning method, meaning that validating the learning results is difficult. However, it is possible to use statistical methods (e.g. silhouette score, which is between -1 and 1) to certify the cluster quality. Silhouette measures how well an object fit within its cluster compared with other clusters (Rousseeuw, 1987).

### 3.2 Linear Regression with VIF

This project adopts a linear regression model with VIF to forecast the crime rate given socio-economic factors relating to income, education and unemployment. Variance Inflation Factor (VIF) is an indicator of multicollinearity, the phenomenon of unexpected high intercorrelations between two or more independent variables. Multicollinearity results in an unstable linear model (i.e., slight data variation can bring abrupt changes in model regression coefficients).

Linear regression is a supervised learning method. We can compare the prediction results with true values in different ways, such as root mean square error and mean absolute error.

## 4. Data Analysis

### 4.1 DBSCAN 

#### 4.1.1 DBSCAN Hyperparameter Tuning

According to a case study (Rahmah and Sitanggang, 2016), an ascending plot of each point's distance to the nearest point can determine an optimal epsilon. The y-axis coordinate of the location, which has a significant slope change, is the suitable epsilon value. The Chicago crime data offers incident locations in latitude and longitude, bringing an extra layer of complexity during the distance calculation. Instead of a simple 2-D Euclidean distance equation, latitude and longitude pairs (spherical coordinates) need to use the haversine formula to find distance values between them. This section implements the haversine formula through the Balltree algorithm to solve the nearest neighbour problem.

The choice of the minPoints value depends on domain knowledge or the dimension of the dataset. In a study of the extended DBSCAN algorithm, network-space DBSCAN (NS-DBSCAN), an explicitly modified DBSCAN for the urban environment, applies MinPts = 20 for clustering (Wang, Ren, Luo and Tian, 2019).

In [ ]:
# parts of code adopted from https://stackoverflow.com/questions/61952561/how-do-i-find-the-neighbors-of-points-containing-coordinates-in-python
DBSCANdf = BURGLARY_2019_spatial

tree = BallTree(
    np.deg2rad(DBSCANdf[["latitude", "longitude"]].values), metric="haversine"
)


DBSCANdf_query = BURGLARY_2019_spatial

qlats = DBSCANdf_query["latitude"]
qlons = DBSCANdf_query["longitude"]

# use k = 2 for 1 closest neighbor (excluding the point itself)
distances, inx = tree.query(np.deg2rad(np.c_[qlats, qlons]), k=2)

distance_list = []
r_km = 6371.0088  # multiplier to convert to km (from unit distance)
for name, d, ind in zip(DBSCANdf_query["id"], distances, inx):
    for i, index in enumerate(ind):
        distance_list.append(d[i] * r_km)

In [ ]:
distance_list = [
    i for i in distance_list if i != 0
]  # remove zeros (distance to the point itself)
distance_list.sort()

In [ ]:
plt.plot(distance_list)

The plot has an 'elbow' when the y value is around 0.35 km. 

#### 4.1.2 DBSCAN Result Visualisation

In [ ]:
kms_per_radian = 6371.0088
epsilon = 0.35 / kms_per_radian
min_samples = 20


def DBfunction(data, epsilon, minsample):
    DBSCAN_result = data
    coords = DBSCAN_result[["latitude", "longitude"]].to_numpy()
    db = DBSCAN(
        eps=epsilon, min_samples=min_samples, algorithm="ball_tree", metric="haversine"
    ).fit(np.radians(coords))
    DBSCAN_result["DBSCAN_opt_labels"] = db.labels_
    DBSCAN_result["DBSCAN_opt_labels"].value_counts()
    return DBSCAN_result


DBSCAN_result = DBfunction(
    data=BURGLARY_2019_spatial, epsilon=epsilon, minsample=min_samples
)
len(DBSCAN_result["DBSCAN_opt_labels"].value_counts())

In [ ]:
def dbPlot(DBSCAN_result):
    DBSCAN_result_wN = DBSCAN_result.drop(
        DBSCAN_result[DBSCAN_result.DBSCAN_opt_labels == -1].index
    )

    fig, ax = plt.subplots(figsize=(10, 10))
    ax.set_xlabel("Longitude")
    ax.set_ylabel("Latitude")
    ax.set_title("Burglary Clusters across the Chicago")

    chicago_community.plot(
        ax=ax, legend=True, column="community_dummy", categorical=True
    )
    chicago_community.apply(  # assign community number on the polygon
        lambda x: ax.annotate(
            text=x["community number"],
            xy=x.geometry.centroid.coords[0],
            ha="center",
            color="#ee8d18",
        ),
        axis=1,
    )
    ax.scatter(  # plot clusters
        DBSCAN_result_wN["longitude"],
        DBSCAN_result_wN["latitude"],
        c=DBSCAN_result_wN["DBSCAN_opt_labels"],
        s=5,
    )

    plt.plot([], [], " ", label="70: community number")
    legend = plt.legend()
    plt.setp(legend.get_texts(), color="#ee8d18")
    score = metrics.silhouette_score(
        DBSCAN_result_wN[["latitude", "longitude"]].to_numpy(),
        DBSCAN_result_wN["DBSCAN_opt_labels"],
        metric="euclidean",
    )
    print("The silhouette score is " + str(score))


dbPlot(DBSCAN_result=DBSCAN_result)

The algorithm spots fifty-four clusters of burglary crime in Chicago. Some areas (e.g. community 25, community 24 and community 67) have multiple clusters, while several northeast regions (e.g. community 17 & 11) and middle sections (e.g. community 56 and 31) have no clusters. 

In [ ]:
# plot the total case number of each cluster
cluster_stats = DBSCAN_result["DBSCAN_opt_labels"].value_counts().to_frame()
cluster_stats = cluster_stats.rename(
    columns={"DBSCAN_opt_labels": "case number in cluster"}
)

cluster_stats = cluster_stats.iloc[1:, :]  # remove the noise row
ax = cluster_stats.sort_index().plot.bar(figsize=(10, 10))
ax.set_xlabel("Cluster index")
ax.set_ylabel("number of cases")
ax.set_title("Number of burglary cases in each cluster")

The graph of the number of cases in each cluster shows that most clusters possess less than one hundred records. However, six clusters have cases exceeding two hundred. The indices of clusters are not arbitrary (i.e. their geolocations can be found on the map).

#### 4.1.3 Qualitative description of DBSCAN Clusters and Discussion

In [ ]:
print(
    "On average, one cluster have "
    + str(DBSCAN_result["DBSCAN_opt_labels"].value_counts().drop(labels=[-1]).mean())
    + " records"
)

The silhouette score (between -1 and 1) is 0.4401230161702594, indicating an acceptable clustering quality, and clusters are not likely to be overlapped or mislabelled. Each cluster, on average, contains near 70 recorded burglary incidents. The clusters are mainly congested into two areas: northern communities around community 24 and southern communities near community 69. Between two regions with a relatively high density of clusters, a buffer zone, namely, community 56-60 and community 33-36, has no cluster. Further in southern Chicago, two clusters scatter over the community 49 and 53, and in the northwest of the city, DBSCAN detects no cluster.

#### 4.1.4 DBSCAN Visualisation With Temporal Constraints

In this section 4.1.2, the visualisation of burglary clusters is complete. However, there is a significant issue undermining the value of the finding, cases inside clusters correlate with each other in the spatial domain, but temporal distances between points are neglected. We must consider the time timestamp for real-world problems (e.g. should a December case be associated with a cluster of multiple January records in the area?). This section assumes a scenario in which the authority requires a crime analysis of the first five months of 2019.

In [ ]:
BURGLARY_2019["date"] = pd.to_datetime(BURGLARY_2019["date"])

BURGLARY_2019 = BURGLARY_2019.sort_values(by=["date"])

mask = (BURGLARY_2019["date"] > "2019-1-1") & (BURGLARY_2019["date"] <= "2019-5-30")
BURGLARY_2019_JAN = BURGLARY_2019.loc[mask]

BURGLARY_2019_JAN_result = DBfunction(
    data=BURGLARY_2019_JAN, epsilon=epsilon, minsample=min_samples
)
dbPlot(DBSCAN_result=BURGLARY_2019_JAN_result)
BURGLARY_2019_JAN_result["DBSCAN_opt_labels"].value_counts()

In [ ]:
print(
    "On average, one cluster have "
    + str(
        BURGLARY_2019_JAN_result["DBSCAN_opt_labels"]
        .value_counts()
        .drop(labels=[-1])
        .mean()
    )
    + " records"
)

A temporal constraint reduces the total number of clusters by 51 (from 54 to 3). The largest cluster in this scenario has 78 cases. During the first five months of 2019, DBSCAN finds one cluster in community 22, another one at the northeast corner of community 44, and a third across the border between community 43 and 46.

#### 4.1.5 DBSCAN Further Work

With one additional hyperparameter, temporal maximum reachable distance, an extended DBSCAN becomes ST-DBSCAN, allowing point data to search neighbouring events in the temporal domain (Chen and Cheng 2020). The rigid temporal constraint in section 4.1.3 may partially solve the temporal correlation issue. Still, it has a border problem (i.e. if the temporal boundary is 1st May, a case that occurred on 2nd May will not associate with any previous cases).

The package st_dbscan is not available in the coursework environment.

### 4.2 Linear Regression with VIF

#### 4.2.1 Variance Inflation Factor

In [ ]:
chicago_community_with_crime_data = chicago_community_with_crime_data[
    ~chicago_community_with_crime_data.isin([np.nan, np.inf, -np.inf]).any(1)
]
chicago_community_with_crime_data = add_constant(chicago_community_with_crime_data)
X_variables = chicago_community_with_crime_data[
    ["unemployment rate", "degree percentage", "median household income"]
]
vif_data = pd.DataFrame()
vif_data["feature"] = X_variables.columns
vif_data["VIF"] = [
    variance_inflation_factor(X_variables.values, i)
    for i in range(len(X_variables.columns))
]

In [ ]:
vif_data

The VIF for median household income is exceptionally higher. To ensure the stability of regression model, further analysis will exclude the median household income.

In [ ]:
chicago_community_with_crime_data = chicago_community_with_crime_data[
    ~chicago_community_with_crime_data.isin([np.nan, np.inf, -np.inf]).any(1)
]
chicago_community_with_crime_data = add_constant(chicago_community_with_crime_data)
X_variables = chicago_community_with_crime_data[
    ["unemployment rate", "degree percentage"]
]
vif_data = pd.DataFrame()
vif_data["feature"] = X_variables.columns
vif_data["VIF"] = [
    variance_inflation_factor(X_variables.values, i)
    for i in range(len(X_variables.columns))
]
vif_data

The VIF for two remaining factors fall into the cceptable range.

#### 4.2.2 Multivariate Linear Regression Model

In [ ]:
regression_dataset = chicago_community_with_crime_data[
    ["incident rate per 1000 people", "unemployment rate", "degree percentage"]
]

In [ ]:
# testing training datasets split
random_state_split = 1999
train_x, test_x, train_y, test_y = train_test_split(
    regression_dataset.drop(["incident rate per 1000 people"], axis=1),
    regression_dataset[["incident rate per 1000 people"]],
    random_state=random_state_split,
)

In [ ]:
# train the regression model
lr = LinearRegression()
lr.fit(X=train_x, y=train_y)

In [ ]:
# test the model
y_pred = lr.predict(test_x)

In [ ]:
# Compare testing values and prediction values
y_pred_frame = pd.DataFrame(y_pred, columns=["pred_rate"])
y_pred_frame
compare = pd.concat(
    [test_y, y_pred_frame.set_index(test_y.index)], axis=1, ignore_index=True
)
compare.columns = ["test_y", "y_pred"]

In [ ]:
# merge prediction of crime rates into community dataframe for plotting
regression_result = pd.merge(
    chicago_community_with_crime_data,
    compare[["y_pred"]],
    left_index=True,
    right_index=True,
)
regression_result.head(1)

#### 4.2.2 Model Peformance

In [ ]:
# give summary info of a linear regression model using statsmodel
regressor_OLS = sm.OLS(endog=train_y, exog=sm.add_constant(train_x)).fit()
regressor_OLS.summary()

Adjusted R-squared equals 0.638, indicating that the linear regression model from section 4.2.2 can explain 63.8% of the change in the dependent variable. The difference between R-squared and adjusted R-squared is that adjusted R-squared penalises the score of a linear regression model according to the number of independent variables. Since a model's R-squared value will never decrease with additional independent variables (investopedia, 2022), adjusted R-squared can better reflect the performance of the burglary rate prediction model.

Another interesting parameter is the P>|t| value or P-value. The P-value for degree percentage (the percentage of the population with a college degree) is 0.267, which means that the probability of this variable being irrelevant during the regression is 26.7%. The coefficient of degree percentage is close to zero for 95% of the training data ([0.025 0.975]: -0.011 0.038). The P-value for the unemployment rate is 0, which suggests a strong relationship between the unemployment rate and the burglary crime rate.

In [ ]:
print(
    "maximum difference between the prediction and the actual value is "
    + str(max((compare["test_y"] - compare["y_pred"]).abs()))
    + "  per 1000 people"
)
print(
    "average difference between the prediction and the actual value is "
    + str((compare["test_y"] - compare["y_pred"]).abs().mean())
    + "  per 1000 people"
)

print(
    "The root mean square error (RMSE) is "
    + str(mean_squared_error(compare["test_y"],  compare["y_pred"], squared=False))
)

#### 4.2.3 Linear Regression Results Visualisation

In [ ]:
ax = (
    compare.rename(
        columns={"test_y": "actual burglary rate", "y_pred": "regression predictions"}
    )
    .reset_index()[["actual burglary rate", "regression predictions"]]
    .plot(style=["o", "rx"])
)
ax.set_xlabel("index")
ax.set_ylabel("burglary rate per 1000 people")
ax.set_title("Actual burglary rate vs prediction rate")

Figure (Actual burglary rate vs prediction rate) reveals that the misfitting between predictions and recorded values vary across the testing dataset. The maximum difference is around 2.74 per one thousand community residents. The average difference between the prediction and the actual value is close to 1 case per 1000 people.

In [ ]:
regression_result_gdf = gpd.GeoDataFrame(
    regression_result, geometry=regression_result.geometry
)
regression_result_gdf = regression_result_gdf.set_crs(
    4326, allow_override=True
)  # set the coordinate reference system for basemap

fig, ax = plt.subplots(figsize=(10, 10))
ax.set_xlabel("Longitude")
ax.set_ylabel("Latitude")
ax.set_title(
    "Burglary Rates of testing group Communities (linear regression's prediction results)"
)
regression_result_gdf.plot(
    ax=ax, legend=True, column="y_pred", categorical=False, alpha=0.8
)
cx.add_basemap(ax, crs=regression_result_gdf.crs)

fig2, ax2 = plt.subplots(figsize=(10, 10))
ax2.set_xlabel("Longitude")
ax2.set_ylabel("Latitude")
ax2.set_title("Burglary Rates of testing group Communities (actual value)")
regression_result_gdf.plot(
    ax=ax2,
    legend=True,
    column="incident rate per 1000 people",
    categorical=False,
    alpha=0.8,
)
cx.add_basemap(ax2, crs=regression_result_gdf.crs)

Some discrepancy exists between prediction and recorded values, but the general crime rate trend remains consistent between the two diagrams.

## 5. Project Summary and Conclusion

This project provides precise locations of burglary crime clusters on a georeferenced map. Law enforcement can consider patrolling these crime cluster areas more often or deploying extra posts to these hotspots. According to relevant literature (Wheeler and Reuter, 2020), installing additional preventive measures around crime hotspots can pay back economically. The figure (Burglary Clusters across the Chicago) is a graphical answer to project question 1 (1. Where should resources be allocated to reduce the burglary crime rate in Chicago?).



Section 4.2 includes the training and testing of the linear regression model that can predict burglary crime rate given specific socio-economic facts of investigation areas. Policymakers can be advised in advance about the possible crime rate trend when the census data is available.

The unsupervised (DBSCAN) and the supervised model (linear regression) of this project are flexible. The DBSCAN model can operate with other crime incidents records as long as they are geocoded in latitude and longitude. With some minor modifications, the linear regression model can handle more columns of demographic datasets.

## 6. Critical Analysis

### 6.1 DBSCAN Critical Analysis

This project's cluster results are immature because, besides the main DBSCAN analysis does not take into account the temporal correlations, the selection of hyperparameters is based on domain knowledge from different discplines other than criminology. The NT-BDSCAN has a promising outlook in urban crime pattern analysis. If we can bring criminology experts and researchers of network clustering together, we may get more accurate clusters that can be of practical use to the police.

### 6.2 Linear Regression Model Critical Analysis

The linear regression model can accommodate continuous variables, but if we need to examine categorical such as if the community has a highway passing through, we need to adopt another regression method. A classification and regression tree (CART) can adapt both discrete and continuous data. However, the project team should carefully consider the choice of datasets since the R-square value (i.e. how much output variance can be explained by model input independent variables) will either increase or stagnate with additional variables. A sophisticated multivariate prediction model requires criminologists and data scientists to work together.

The selection of census datasets follows studies of different cities and areas. The variation in the cultural, political and economic context of study areas may cause correlations less suitable for investigating the Chicago crime data. Before the regression study, a separate analysis of the relationship between the Chicago social-economic background and crime rate may be needed.

# Bibliography

Andresen, M., 2005. Crime Measures and the Spatial Analysis of Criminal Activity. The British Journal of Criminology, 46(2), pp.258-285.

Charron, M. 2009. Neighbourhood Characteristics and the Distribution of Police-reported Crime in the City of Toronto. Canadian Centre For Justice Statistics. Catalogue no. 85-561-M, no. 18

Craglia, M., Haining, R. and Wiles, P., 2000. A Comparative Evaluation of Approaches to Urban Crime Pattern Analysis. Urban Studies, 37(4), pp.711-729.

Ester, M., Kriegel, H.-P., Sander, J., & Xu, X. (1996). A density-based algorithm for discovering clusters in large spatial databases with noise. 226–231.

FBI UCR. 2019. Burglary. [online] Available at: <https://ucr.fbi.gov/crime-in-the-u.s/2019/crime-in-the-u.s.-2019/topic-pages/burglary> [Accessed 17 March 2022].

Hjalmarsson, Randi & Lochner, Lance. (2012). The Impact of Education on Crime: International Evidence. CESifo Dice Report, Journal for Institutional Comparisons. 10. 

Investopedia. 2022. What Is R-Squared?. [online] Available at: <https://www.investopedia.com/terms/r/r-squared.asp> [Accessed 23 April 2022].

Lin, M., 2008. Does Unemployment Increase Crime?: Evidence from U.S. Data 1974-2000. Journal of Human Resources, 43(2), pp.413-436.

Mungekar, D., Joshi, H., Kankekar, A., Nair, P. and Das, P., 2021. Crime Analysis using DBSCAN Algorithm. 2021 Third International Conference on Inventive Research in Computing Applications (ICIRCA),.

Rahmah, N. and Sitanggang, I., 2016. Determination of Optimal Epsilon (Eps) Value on DBSCAN Algorithm to Clustering Data on Peatland Hotspots in Sumatra. IOP Conference Series: Earth and Environmental Science, 31, p.012012.

Roshan P. Thakur, 2020. CRIME ANALYSIS AND PREDICTION - BY USING DBSCAN ALGORITHM. International Research Journal of Engineering and Technology (IRJET), 7(4), pp. 4262 - 4266

Rousseeuw, P., 1987. Silhouettes: A graphical aid to the interpretation and validation of cluster analysis. Journal of Computational and Applied Mathematics, 20, pp.53-65.

Sivaranjani, S., Sivakumari, S. and Aasha, M., 2016. Crime prediction and forecasting in Tamilnadu using clustering approaches. 2016 International Conference on Emerging Technological Trends (ICETT),.

Wang, T., Ren, C., Luo, Y. and Tian, J., 2019. NS-DBSCAN: A Density-Based Clustering Algorithm in Network Space. ISPRS International Journal of Geo-Information, 8(5), p.218.

Wheeler, A. and Reuter, S., 2020. Redrawing Hot Spots of Crime in Dallas, Texas. Police Quarterly, 24(2), pp.159-184.
